In [9]:
import calcbench as cb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import numpy as np
from  time import time
import sklearn
from sklearn import metrics
from scipy.spatial.distance import cosine
from IPython.core.display import display, HTML

In [2]:
class NumberNormalizingVectorizer(sklearn.feature_extraction.text.TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super(NumberNormalizingVectorizer, self).build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

In [3]:
def number_normalizer(tokens):
    """ Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """

    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

In [4]:
tickers = cb.tickers(index='DJIA')

In [11]:
similarities = []
for ticker in tickers:
    risks_2016 = next(cb.document_search(company_identifiers=[ticker], document_name='Risk Factors', year=2016)).get_contents()
    risks_2015 = next(cb.document_search(company_identifiers=[ticker], document_name='Risk Factors', year=2015)).get_contents()
    text_2016 = BeautifulSoup(risks_2016, 'html.parser').text
    text_2015 = BeautifulSoup(risks_2015, 'html.parser').text
    vectorizer = NumberNormalizingVectorizer(stop_words='english')
    X = vectorizer.fit_transform([text_2016, text_2015])
    distance =  cosine(X[0].todense(), X[1].todense())
    similarities.append({'ticker' : ticker, '2016' : risks_2016, '2015' : risks_2015, 'distance' : distance})
                       

c:\users\andrew kittredge\appdata\local\programs\python\python35\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [26]:
most_different_document = sorted(similarities, key=lambda d: d['distance'], reverse=True)[1]

In [27]:
most_different_document['ticker']

'V'

In [28]:
display(HTML(cb.html_diff(most_different_document['2016'], most_different_document['2015'])))